In [1]:
import tensorflow as tf

In [4]:
#Include two different sections:
#1. Build computational graph
#2. Run computational graph
node1 = tf.constant(3.0, dtype = tf.float32)
node2 = tf.constant(4.0)

print(node1, node2)

#Evaluate node within a session to gain output
sess = tf.Session()
print(sess.run([node1, node2]))

#Create new node throught operation
node3 = tf.add(node1, node2)
print('node 3:', node3)
print(sess.run([node3]))

Tensor("Const_4:0", shape=(), dtype=float32) Tensor("Const_5:0", shape=(), dtype=float32)
[3.0, 4.0]
node 3: Tensor("Add:0", shape=(), dtype=float32)
[7.0]


In [8]:
#contrast with constant which only take constant values, placeholder promise to get any values provide to it later
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

adder_node = a + b #short cut for operation tf.add(a,b)

#using session.run(node_operation, feed_dict), feed_dict will feed node operation by it parameter palaceholder. 

print(sess.run(adder_node, {a:3, b:4}))
print(sess.run(adder_node, {a:[1, 2], b:[4, 5]}))

add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a:3, b:4.5}))

7.0
[5. 7.]
22.5


In [9]:
#Variable allow us to train parameter in computational graph
W = tf.Variable([.3], dtype = tf.float32)
b = tf.Variable([-.3], dtype = tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

#to initalized value to Variable we have to call tf.global_variables_initializer()
init = tf.global_variables_initializer()
sess.run(init)

In [13]:
#after we init parameter throw init, we could evaluate model by set value to placeholder
print(sess.run(linear_model, {x: [1,2,3,4]}))

[0.         0.3        0.6        0.90000004]


In [15]:
#create loss function and evaluate loss function
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
sess.run(loss, {x:[1,2,3,4], y:[-1,-2,-3,-4]})

43.26

In [18]:
#reassign to Variable W and b to get new return loss function values
fix_W = tf.assign(W, [-1])
fix_b = tf.assign(b, [0])
sess.run([fix_W, fix_b])
sess.run(loss, {x:[1,2,3,4], y:[-1,-2,-3,-4]})

0.0

In [19]:
#train API by optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

#reset values to incorrect defaults
sess.run(init)
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[-1,-2,-3,-4]})
print(sess.run([W, b]))    

[array([-0.9999987], dtype=float32), array([-3.871269e-06], dtype=float32)]


In [21]:
#tf.contrib.learn is a high-level Tensorflow lib that simplified mechanics of machine learning:
#1. training loop
#2. eval loop
#3. manage datasets
#4. manage feeding

import tensorflow as tf
import numpy as np

#declare list of features
features = [tf.contrib.layers.real_valued_column('x', dimension = 1)]

#define type of training like linear, logistic, neural network,....
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.contrib.learn.io.numpy_input_fn(
    {"x":x_train}, y_train, 
    batch_size = 4,
    num_epochs = 1000
)


eval_input_fn = tf.contrib.learn.io.numpy_input_fn(
    {"x":x_eval}, y_eval,
    batch_size = 4,
    num_epochs = 1000
)

# We invoke 1000 training steps by invoking method and passing to the training data set
estimator.fit(input_fn = input_fn, steps = 1000)

#evaluate model
train_loss = estimator.evaluate(input_fn = input_fn)
eval_loss = estimator.evaluate(input_fn = eval_input_fn)

print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E5E1CF4EB8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\LAPTOP~1\\AppData\\Local\\Temp\\tmps3gmpkod'}
Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\LAPTOP~1\AppData\Local\Temp\tmps3gmpkod\model.ckpt.
INFO:tensorflow:loss = 0.5, step = 1
INFO:tensorflow:global

# Custom model which is not builed in tf.contrib.learn

In [22]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, 4, num_epochs=1000)

# train
estimator.fit(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did. 
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E5E01E6E10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\LAPTOP~1\\AppData\\Local\\Temp\\tmp3kz8h10q'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\LAPTOP~1\AppData\Local\Temp\tmp3kz8h10q\model.ckpt.
INFO:tensorflow:loss = 90.37633447347056, step = 1
INFO:tensorflow:global_step/sec: 379.117
INFO:tensorflow:loss = 0.00129689909

In [6]:
import numpy as np
X = np.array([[1],
              [2],
              [3]])
X.shape
Y = np.array([[1,2,3]])
Y.shape
X.dot(Y)
Y.dot(X)

array([[14]])